In [1]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *
cc.setupEnvironment()
cc.listEnvironment()

ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 2120
JPY_INTERRUPT_EVENT: 2120
JPY_PARENT_PID: 2124
JPY_SESSION_NAME: DWH_dimUser_initial.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LO

In [2]:
spark = cc.startLocalCluster("dimUsersInit")
spark.getActiveSession()

In [3]:
cc.set_connectionProfile("veloDB")

df_operational_users = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

df_operational_users.createOrReplaceTempView("operational_users")


In [4]:
# Create a temporary view for operational users
df_operational_users.createOrReplaceTempView("dimUsers")

# Add surrogate key, start date, end date, md5, and current flag
df_user_dim_current = spark.sql("""
    SELECT 
        uuid() AS userSK,  -- Surrogate key
        userid AS source_user_id,  -- User ID
        name AS source_name,  -- User Name
        city AS source_city,  -- User City
        zipcode AS source_zipcode,  -- User Zipcode
        to_timestamp('1999-01-01', 'yyyy-MM-dd') AS scd_start,  -- Start date
        to_timestamp('2100-12-12', 'yyyy-MM-dd') AS scd_end,  -- End date (far future)
        md5(concat(name, city, zipcode)) AS source_md5,  -- MD5 hash to detect changes
        TRUE AS current  -- Mark the current record
    FROM operational_users
""")

# Save as a Delta table (first time creation)
df_user_dim_current.write.format("delta").mode("overwrite").saveAsTable("user_dim_current")


In [5]:
# Describe the Delta table and find the location
spark.sql("DESCRIBE FORMATTED user_dim_current").show(truncate=False)


+----------------------------+-----------------------------------------------------------------+-------+
|col_name                    |data_type                                                        |comment|
+----------------------------+-----------------------------------------------------------------+-------+
|userSK                      |string                                                           |NULL   |
|source_user_id              |int                                                              |NULL   |
|source_name                 |string                                                           |NULL   |
|source_city                 |string                                                           |NULL   |
|source_zipcode              |string                                                           |NULL   |
|scd_start                   |timestamp                                                        |NULL   |
|scd_end                     |timestamp                